In [2]:
import pandas as pd
import numpy as np
import os


#os.chdir("C:/Users/cuent/En-Peu/notebooks/")
print(os.getcwd())

C:\Users\cuent\En-Peu\notebooks


In [3]:
# Cambiar al directorio padre
os.chdir("..")
parent_directory = os.getcwd()

# os.path.join() maneja correctamente las barras de ruta para diferentes SO (Windows, Linux, macOS)
file_path = os.path.join(parent_directory, 'data', 'data_stored.csv')

# Cargar el archivo CSV
data = pd.read_csv(file_path)


# Verificar la estructura del DataFrame (equivalente a str(data) en R)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64345 entries, 0 to 64344
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   POINT_X  64345 non-null  float64
 1   POINT_Y  64345 non-null  float64
 2   PLA      64345 non-null  float64
 3   A        64345 non-null  float64
 4   AC       64345 non-null  float64
 5   CIB      64345 non-null  float64
 6   CICCB    64345 non-null  float64
 7   CUS      64345 non-null  int64  
 8   DBB      64345 non-null  float64
 9   DOB      64345 non-null  float64
 10  OC       64345 non-null  int64  
 11  P        64345 non-null  float64
dtypes: float64(10), int64(2)
memory usage: 5.9 MB
None


In [4]:
# Podemos ver que CUS y OC no se están contando como categóricas. Hay que transformarlas
data['CUS'] = data['CUS'].astype('category')
data['OC'] = data['OC'].astype('category')
# Verificar la estructura del DataFrame (equivalente a str(data) en R)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64345 entries, 0 to 64344
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   POINT_X  64345 non-null  float64 
 1   POINT_Y  64345 non-null  float64 
 2   PLA      64345 non-null  float64 
 3   A        64345 non-null  float64 
 4   AC       64345 non-null  float64 
 5   CIB      64345 non-null  float64 
 6   CICCB    64345 non-null  float64 
 7   CUS      64345 non-null  category
 8   DBB      64345 non-null  float64 
 9   DOB      64345 non-null  float64 
 10  OC       64345 non-null  category
 11  P        64345 non-null  float64 
dtypes: category(2), float64(10)
memory usage: 5.0 MB
None


In [5]:
# --- Identificar variables numéricas excluyendo las coordenadas ---
# Lista de columnas que se consideran coordenadas espaciales
coordenadas = ['POINT_X', 'POINT_Y']

# Seleccionar todas las columnas numéricas
numericas = data.select_dtypes(include=np.number).columns.tolist()

# Filtrar las columnas numéricas para excluir las coordenadas
# Usamos un conjunto para una búsqueda eficiente de coordenadas
cols_to_scale = [col for col in numericas if col not in set(coordenadas)]

print(f"\nColumnas a escalar (numéricas excluyendo coordenadas): {cols_to_scale}")


Columnas a escalar (numéricas excluyendo coordenadas): ['PLA', 'A', 'AC', 'CIB', 'CICCB', 'DBB', 'DOB', 'P']


In [6]:
from sklearn.preprocessing import MinMaxScaler

# Usar el MinMaxScaler de sklearn
scaler = MinMaxScaler()
# calcular min/max y escalar
data[cols_to_scale] = scaler.fit_transform(data[cols_to_scale])

# Guardar el objeto para poder tener el min/max en el futuro
import joblib
joblib.dump(scaler, os.path.join(parent_directory, 'data','min_max_scaler.pkl'))
# To load later: 
# loaded_scaler = joblib.load(os.path.join(parent_directory, 'data','min_max_scaler.pkl'))
# data_new_scaled = loaded_scaler.transform(data_new[cols_to_scale])
print("Data después de usar sklearn MinMaxScaler:")
print(data.head())

Data después de usar sklearn MinMaxScaler:
         POINT_X       POINT_Y  PLA         A        AC       CIB     CICCB  \
0  722396.507981  4.365860e+06  0.0  0.604254  0.067778  0.203910  0.390004   
1  722404.245816  4.365860e+06  0.0  0.581583  1.000000  0.201362  0.388246   
2  722411.983652  4.365860e+06  0.0  0.581583  1.000000  0.198815  0.386458   
3  722396.502336  4.365868e+06  0.0  0.609629  1.000000  0.204328  0.391055   
4  722404.240164  4.365868e+06  0.0  0.608561  1.000000  0.201762  0.389291   

  CUS       DBB       DOB OC         P  
0   5  0.620621  0.000000  4  0.077332  
1   1  0.614835  0.788748  4  0.089864  
2   1  0.610494  0.783177  4  0.089864  
3   1  0.620342  0.000000  4  0.051674  
4   1  0.614553  0.788748  4  0.062030  


Ahora hay que transformar las variables categóricas con One Hot Encoder

In [7]:
from sklearn.preprocessing import OneHotEncoder

categorical_cols = ['CUS', 'OC']

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse_output=False for dense array, handle_unknown='ignore' for new categories

# Fit and transform the categorical data
encoded_features = encoder.fit_transform(data[categorical_cols])

# Create a DataFrame from the encoded features
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))

# Drop original columns and concatenate new encoded columns
data_encoded_sklearn = data.drop(columns=categorical_cols)
data_encoded_sklearn = pd.concat([data_encoded_sklearn, encoded_df], axis=1)

print("\nData after sklearn OneHotEncoder:")
print(data_encoded_sklearn.head())


Data after sklearn OneHotEncoder:
         POINT_X       POINT_Y  PLA         A        AC       CIB     CICCB  \
0  722396.507981  4.365860e+06  0.0  0.604254  0.067778  0.203910  0.390004   
1  722404.245816  4.365860e+06  0.0  0.581583  1.000000  0.201362  0.388246   
2  722411.983652  4.365860e+06  0.0  0.581583  1.000000  0.198815  0.386458   
3  722396.502336  4.365868e+06  0.0  0.609629  1.000000  0.204328  0.391055   
4  722404.240164  4.365868e+06  0.0  0.608561  1.000000  0.201762  0.389291   

        DBB       DOB         P  CUS_1  CUS_2  CUS_3  CUS_4  CUS_5  OC_1  \
0  0.620621  0.000000  0.077332    0.0    0.0    0.0    0.0    1.0   0.0   
1  0.614835  0.788748  0.089864    1.0    0.0    0.0    0.0    0.0   0.0   
2  0.610494  0.783177  0.089864    1.0    0.0    0.0    0.0    0.0   0.0   
3  0.620342  0.000000  0.051674    1.0    0.0    0.0    0.0    0.0   0.0   
4  0.614553  0.788748  0.062030    1.0    0.0    0.0    0.0    0.0   0.0   

   OC_2  OC_3  OC_4  
0   0.0   0

In [9]:
# Guardar el resultado en un nuevo CSV
nombre_archivo_salida = "data_fin.csv"
data_encoded_sklearn.to_csv(os.path.join(parent_directory,'data',nombre_archivo_salida), index=False) # index=False evita guardar el índice de pandas como una columna

print(f"\nDataFrame combinado guardado exitosamente como '{nombre_archivo_salida}' en '{os.path.join(parent_directory,'data',nombre_archivo_salida)}'")


DataFrame combinado guardado exitosamente como 'data_fin.csv' en 'C:\Users\cuent\En-Peu\data\data_fin.csv'
